In [29]:
#Library/Packages

import os
import xml.etree.ElementTree as ET
from lxml import etree
from tqdm import tqdm
import logging
from datetime import datetime
import xml.dom.minidom as md
import importlib
import db
importlib.reload(db)
# Configure logging
logFile = datetime.now().strftime('logfile_%H_%M_%d_%m_%Y.log')
#print(logFile)
logging.basicConfig(filename=logFile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [43]:


#df_isicily = db.run_query("Select * from iSicily")
#print(df_isicily)

2024-07-23 10:43:02,589| ERROR   | Password is required for key /Users/imranasif/.ssh/id_rsa
2024-07-23 10:43:02,592| ERROR   | Password is required for key /Users/imranasif/.ssh/id_ed25519


### Common Methods

In [80]:
db_cursor = None
namespaces = {
    'tei': 'http://www.tei-c.org/ns/1.0'  # Replace with the actual namespace URI
}

def getFileName(file_path):
    # Get the base name (filename with extension)
    base_name = os.path.basename(file_path)

    # Split the base name into name and extension
    name, ext = os.path.splitext(base_name)
    return name

def extractDatesFromXMLBySource(xml_dir, source, projectURL, tableName, xmlDateBefore, xmlDateAfter):
    for filename in tqdm(os.listdir(xml_dir), desc='Processing', unit='item'):
        try:
            if filename.endswith('.xml'):
                xml_path = os.path.join(xml_dir, filename)
                
                dateDict = {
                    'dateText': None,
                    'datingMethod': None,
                    'notBeforeCustom': None,
                    'notAfterCustom': None,
                    'evidence': None,
                    'Id': None,
                    'source': None,
                    'projectURL': None
                }

                #print(xml_path)
                # Parse the XML file
                # Define the XML parser with recovery mode enabled
                parser = etree.XMLParser(recover=True)

                tree = etree.parse(xml_path, parser)
                root = tree.getroot()
                
                # Find all <ptr> child elements
                origDate_elements = root.findall('.//tei:origDate', namespaces)
                
                if len(origDate_elements) == 0:
                    logging.info(f'There is no origDate element available in xml file: {filename}')
                    #print(dateDict)
                    #continue

                # if origDate_elements[0].get('evidence') == None:
                #     logging.info(f'There is no evidence element available in xml file: {filename}')
                #     continue

                #print(ptr_elements)
                # Process the <ptr> elements
                for date_element in origDate_elements:
                    
                    dateDict['dateText'] = date_element.text

                    if date_element.get('datingMethod') != None:
                        dateDict['datingMethod'] = date_element.get('datingMethod')
                    else:
                        logging.info(f'There is no datingMethod attribute in origDate element available in xml file: {filename}')

                    if date_element.get(xmlDateBefore) != None:
                        dateDict['notBeforeCustom'] = date_element.get(xmlDateBefore)
                    else:
                        logging.info(f'There is no notBefore-custom attribute in origDate element available in xml file: {filename}')
                    
                    if date_element.get(xmlDateAfter) != None:
                        dateDict['notAfterCustom'] = date_element.get(xmlDateAfter)
                    else:
                        logging.info(f'There is no notAfter-custom attribute in origDate element available in xml file: {filename}')

                    if date_element.get('evidence') != None:
                        dateDict['evidence'] = date_element.get('evidence')
                    else:
                        logging.info(f'There is no evidence attribute in origDate element available in xml file: {filename}')

                dateDict['Id'] = getFileName(filename)
                dateDict['source'] = source
                dateDict['projectURL'] = projectURL
                #print(dateDict)
                insert(dateDict, tableName)
                #break

        except ET.ParseError as e:
           logging.error(f'Exception: {str(e)} in xml file {filename}')
        except db.pymysql.MySQLError as e:
            logging.error(f'Exception: {str(e)} in xml file {filename}')
            break
        except Exception as e:
            #logging.info(f'Issue in xml file: {filename}')
            logging.error(f'Exception: {str(e)} in xml file {filename}')


def insert(data, tableName):
    sql = f'''INSERT INTO `{tableName}` (Id, dateText, datingMethod, notBeforeCustom, notAfterCustom, evidence, source, projectURL)
                     VALUES(%s,%s,%s,%s,%s,%s,%s,%s)
                 '''
    db_cursor.execute(sql, (data['Id'], data['dateText'], data['datingMethod'], data['notBeforeCustom'], data['notAfterCustom'], data['evidence'], data['source'], data['projectURL']))
    
    db.connection.commit()

def openConnection():
    db.open_ssh_tunnel()
    db.mysql_connect()
    global db_cursor
    db_cursor = db.connection.cursor()

def closeConnection():
    db.mysql_disconnect()
    db.close_ssh_tunnel()

### Collecting Dating Data from IScily

In [45]:
xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/ISicily'
extractDatesFromXMLBySource(xml_dir, 'I.Sicily', 'http://sicily.classics.ox.ac.uk/', 'iSicily', 'notBefore-custom', 'notAfter-custom')

Processing: 100%|██████████| 4722/4722 [02:46<00:00, 28.34item/s]


### Collecting Dating Data from Inscriptions of Greek Cyrenaica (IGCYR) - Greek Verse Inscriptions of Cyrenaica (GVCYR)

In [53]:
openConnection()

2024-07-23 12:17:57,267| ERROR   | Password is required for key /Users/imranasif/.ssh/id_rsa
2024-07-23 12:17:57,269| ERROR   | Password is required for key /Users/imranasif/.ssh/id_ed25519


In [54]:

xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/IGCYR-GVCYR'
extractDatesFromXMLBySource(xml_dir, 'IGCYR-GVCYR', 'https://igcyr.unibo.it/', 'IGCYR-GVCYR', 'notBefore', 'notAfter')

Processing: 100%|██████████| 974/974 [00:35<00:00, 27.67item/s]


In [75]:
closeConnection()

### Collecting Dating Data from RIB

In [85]:
openConnection()

2024-07-23 14:06:30,294| ERROR   | Password is required for key /Users/imranasif/.ssh/id_rsa
2024-07-23 14:06:30,298| ERROR   | Password is required for key /Users/imranasif/.ssh/id_ed25519


In [70]:
#Volume 1
xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/RIB/rib-vol-1'
extractDatesFromXMLBySource(xml_dir, 'RIB VOLUME 1', 'https://romaninscriptionsofbritain.org/inscriptions/', 'RIB', 'notBefore-custom', 'notAfter-custom')

Processing: 100%|██████████| 2401/2401 [01:21<00:00, 29.47item/s]


In [82]:
#Volume 2
xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/RIB/rib-vol-2'
extractDatesFromXMLBySource(xml_dir, 'RIB VOLUME 2', 'https://romaninscriptionsofbritain.org/inscriptions/', 'RIB', 'notBefore-custom', 'notAfter-custom')

Processing: 100%|██████████| 10259/10259 [06:36<00:00, 25.87item/s]


In [86]:
#Volume 3
xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/RIB/rib-vol-3'
extractDatesFromXMLBySource(xml_dir, 'RIB VOLUME 3', 'https://romaninscriptionsofbritain.org/inscriptions/', 'RIB', 'notBefore-custom', 'notAfter-custom')

Processing: 100%|██████████| 550/550 [00:21<00:00, 25.05item/s]


In [87]:
closeConnection()

### Collecting Dating Data from EDH

In [88]:
openConnection()

2024-07-23 14:16:03,699| ERROR   | Password is required for key /Users/imranasif/.ssh/id_rsa
2024-07-23 14:16:03,703| ERROR   | Password is required for key /Users/imranasif/.ssh/id_ed25519


In [89]:
xml_dir = '/Users/imranasif/Oxford/FAIR_Epigraphy/DatedAncientInscriptions/data/EDH'
extractDatesFromXMLBySource(xml_dir, 'EDH', 'https://edh.ub.uni-heidelberg.de/edh/inschrift/', 'EDH', 'notBefore-custom', 'notAfter-custom')

Processing: 100%|██████████| 81883/81883 [54:01<00:00, 25.26item/s]  


In [90]:
closeConnection()